<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/SBShim/DKTC_KcELECTRA_SB_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# import tensorflow as tf
# import os

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

# strategy = tf.distribute.TPUStrategy(resolver)

In [4]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [5]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# import numpy as np

# # 한글 지원 폰트
# plt.rc('font', family='NanumBarunGothic')

# # 마이너스 부호 
# plt.rcParams['axes.unicode_minus'] = False

In [6]:
# !pip install transformers

In [7]:
#! pip install datasets

In [8]:
# # Mecab 설치
# !apt-get update
# !apt-get install g++ openjdk-8-jdk 
# !pip3 install konlpy JPype1-py3
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tqdm
from transformers import Trainer, TrainingArguments
import re
import pyarrow as pa
from datasets import Dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.model_selection import train_test_split
from datasets import load_metric

## 데이터 불러오기

In [10]:
data = pd.read_csv('/content/drive/MyDrive/공부/AIFFELTHON/data/train.csv')
data.head(10)

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
5,5,직장 내 괴롭힘 대화,나 이틀뒤에 가나다 음식점 예약좀 해줘. 저녁7시로.\n가나다 음식점이요.?\n응....
6,6,기타 괴롭힘 대화,35번 손님 아이스커피 두잔나왔습니다\n아이스커피? \n네 맛있게드세요\n저기요 아...
7,7,기타 괴롭힘 대화,경비아저씨 내차에 경고장 붙였어요?\n내가 여기 몇년째 사는데 아직도 내차인줄 모르...
8,8,기타 괴롭힘 대화,이거 할인 된다면서요\n 확인해보겠습니다\n 아까 저 시람은 할인 해주던데\n 네 ...
9,9,갈취 대화,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...


## 데이터 전처리

In [11]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(data['class'])
num_labels = len(label_encoder.classes_)

data['label'] = np.asarray(label_encoder.transform(data['class']), dtype=np.int32)
data.head(10)

,idx,class,conversation,label
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,3
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,3
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,1
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,0
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,0
5,5,직장 내 괴롭힘 대화,나 이틀뒤에 가나다 음식점 예약좀 해줘. 저녁7시로.\n가나다 음식점이요.?\n응....,2
6,6,기타 괴롭힘 대화,35번 손님 아이스커피 두잔나왔습니다\n아이스커피? \n네 맛있게드세요\n저기요 아...,1
7,7,기타 괴롭힘 대화,경비아저씨 내차에 경고장 붙였어요?\n내가 여기 몇년째 사는데 아직도 내차인줄 모르...,1
8,8,기타 괴롭힘 대화,이거 할인 된다면서요\n 확인해보겠습니다\n 아까 저 시람은 할인 해주던데\n 네 ...,1
9,9,갈취 대화,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...,0


In [12]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report

# # 대화와 라벨 분류
# train = data['conversation'].tolist()
# target = data['label'].tolist()

# # train, val 분리 (20%)
# train_x, val_x, train_y, val_y = train_test_split(train, target, random_state=27, test_size=0.2)

## Tokenizer 및 Mdoel 불러오기

In [14]:
from transformers import AutoTokenizer, AutoModel
from transformers import ElectraTokenizer, ElectraForSequenceClassification

HUGGINGFACE_MODEL_PATH = "beomi/KcELECTRA-base"

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
#model = AutoModel.from_pretrained("beomi/KcELECTRA-base")
model = ElectraForSequenceClassification.from_pretrained("beomi/KcELECTRA-base", num_labels=num_labels)

# train_tensor = tokenizer(train_x, truncation=True, padding=True)
# val_tensor = tokenizer(val_x, truncation=True, padding=True)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.ou

In [ ]:
# # trainset-set
# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(train_tensor),
#     train_y
# ))

# # validation-set
# val_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(val_tensor),
#     val_y
# ))

## 학습을 위한 데이터셋 생성

In [15]:
import pyarrow as pa
from datasets import Dataset

data = Dataset(pa.Table.from_pandas(data))

In [16]:
def transform(data):
  return tokenizer(
      data['conversation'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_data = data.map(transform, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [17]:
train_dataset, validation_dataset= encoded_data.train_test_split(test_size=0.1).values()

In [18]:
train_dataset

Dataset({
    features: ['idx', 'class', 'conversation', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3555
})

In [22]:
metric = load_metric("f1")

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels, average="micro")

output_dir = '/content/drive/MyDrive/공부/AIFFELTHON/etc'

training_arguments = TrainingArguments(
    output_dir, 
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, 
    per_device_train_batch_size = 8, 
    per_device_eval_batch_size = 8, 
    num_train_epochs = 3,
    weight_decay = 0.01
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
trainer = Trainer(
    model=model,                           
    args=training_arguments,                  
    train_dataset=train_dataset,    
    eval_dataset=validation_dataset,      
    compute_metrics=compute_metrics
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: conversation, class, idx. If conversation, class, idx are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3555
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1335
  Number of trainable parameters = 124548100


Epoch,Training Loss,Validation Loss,F1
1,No log,0.298107,0.913924
2,0.637000,0.310038,0.926582
3,0.232500,0.331681,0.926582


The following columns in the evaluation set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: conversation, class, idx. If conversation, class, idx are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 395
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-500
Configuration saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: conversation, class, idx. If conversation, class, idx are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num exampl

TrainOutput(global_step=1335, training_loss=0.35899060424347495, metrics={'train_runtime': 1079.6768, 'train_samples_per_second': 9.878, 'train_steps_per_second': 1.236, 'total_flos': 2806129794723840.0, 'train_loss': 0.35899060424347495, 'epoch': 3.0})

In [ ]:
# with strategy.scope():
#     model = TFBertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=num_labels, from_pt=True)
#     optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
#     model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['f1'])
#model = TFBertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels = num_labels)

In [ ]:
# trainer = Trainer(
#     model=model,                           
#     args=training_arguments,                  
#     train_dataset=train_dataset,    
#     eval_dataset=val_dataset,      
#     compute_metrics=compute_metrics
# )

In [ ]:
# with strategy.scope():
#     trainer.train()

In [ ]:
# history = model.fit(
#     train_dataset.shuffle(len(train_dataset)).batch(8), epochs=3, batch_size=8,
#     validation_data=val_dataset.shuffle(len(val_dataset)).batch(8)
# )

In [ ]:
# id2labels = model.config.id2label
# model.config.id2label = {id : label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0]  for id, label in id2labels.items()}

# label2ids = model.config.label2id
# model.config.label2id = {label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0] : id   for id, label in id2labels.items()}

In [24]:
import os

DIR_PATH = '/content/drive/MyDrive/공부/AIFFELTHON/model'
MODEL_NAME = 'fine-tuned-KcELECTRA-base'
MODEL_SAVE_PATH = os.path.join(DIR_PATH, MODEL_NAME) # change this to your preferred location

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

Configuration saved in /content/drive/MyDrive/공부/AIFFELTHON/model/fine-tuned-KcELECTRA-base/config.json


/content/drive/MyDrive/공부/AIFFELTHON/model/fine-tuned-KcELECTRA-base -- Folder create complete 



Model weights saved in /content/drive/MyDrive/공부/AIFFELTHON/model/fine-tuned-KcELECTRA-base/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/공부/AIFFELTHON/model/fine-tuned-KcELECTRA-base/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/공부/AIFFELTHON/model/fine-tuned-KcELECTRA-base/special_tokens_map.json


('/content/drive/MyDrive/공부/AIFFELTHON/model/fine-tuned-KcELECTRA-base/tokenizer_config.json',
 '/content/drive/MyDrive/공부/AIFFELTHON/model/fine-tuned-KcELECTRA-base/special_tokens_map.json',
 '/content/drive/MyDrive/공부/AIFFELTHON/model/fine-tuned-KcELECTRA-base/vocab.txt',
 '/content/drive/MyDrive/공부/AIFFELTHON/model/fine-tuned-KcELECTRA-base/added_tokens.json',
 '/content/drive/MyDrive/공부/AIFFELTHON/model/fine-tuned-KcELECTRA-base/tokenizer.json')